In [1]:
import numpy as np
import os
import librosa
import librosa.display
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt


from tensorflow import keras


place all the files from training folder into train_files

In [109]:
def getFiles(data, labels, direct):
    pathList = os.listdir(direct)
    for l in range(len(pathList)):
        if ".flac" in pathList[l]:
            data.append(direct + '/' +  pathList[l])
        elif ".txt" in pathList[l]:
            labels.append(direct + '/' +  pathList[l])
        else:
            temp = direct + '/' +  pathList[l]
            getFiles(data, labels, temp)
    return(data,labels)



In [111]:

t_data = []
t_label = []
test_data = []
test_label = []

train_path = 'Train'
test_path = 'Test'

t_data, t_label = getFiles(t_data,t_label ,train_path)
test_data, test_label = getFiles(test_data,test_label ,test_path)

print(len(t_data))
print(len(t_label))
print(len(test_data))
print(len(test_label))

print(t_data[3])
print(test_data[3])

132
33
132
33
Train/Sound/116-288045-0003.flac
Test/Sound/1688-142285-0003.flac


In [112]:
def spliting(procent, data):
    length = len(data)
    split = int(np.floor(length*procent))
    data1 = data[:split]
    data2 = data[split:]
    return data1,data2

train_data, val_data  = spliting(0.8,t_data)
train_label, val_label  = spliting(0.8,t_label)

print(len(val_data), len(train_data))
print(len(val_label), len(train_label))



27 105
7 26


In [122]:
def changeArray(array):
    array = toDataFrame(array)
    array = addColumn(array)
    return array
    
def toDataFrame(array):
    array = pd.DataFrame(array)
    array = array.rename(columns={0:'file'})
    return array

def addColumn(array):
    speaker = []
    for i in range(0, len(array)):
        speaker.append(array['file'][i].split('-')[0])
        speaker[i] = speaker[i].split('/')[-1]
    # We now assign the speaker to a new column 
    array['speaker'] = speaker
    return array



In [124]:
train_data = changeArray(train_data)
train_label= changeArray(train_label)
val_data = changeArray(val_data)
val_label = changeArray(val_label)
test_data = changeArray(test_data)
test_label = changeArray(test_label)
print(train_data['speaker'][3])

116


In [179]:
def testFiles(files):
    # Sets the name to be the path to where the file is in my computer
    file_name = str(files.file)
    # Loads the audio file as a floating point time series and assigns the default sample rate
    # Sample rate is set to 22050 by default
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')                                #May need to test kaiser_fast
    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    stft = np.abs(librosa.stft(X))
    stft = np.abs(librosa.stft(X))
    # Computes a chromagram from a waveform or power spectrogram.
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    # Computes spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    # Computes the tonal centroid features (tonnetz)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs, chroma, contrast, tonnetz

In [180]:
train_features= train_data.apply(testFiles, axis =1)
val_features=val_data.apply(testFiles, axis =1)
test_features= test_data.apply(testFiles, axis =1)

E:\skola\code\New folder\envs\ann2test\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=815
  return f(*args, **kwargs)
E:\skola\code\New folder\envs\ann2test\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=734
  return f(*args, **kwargs)
E:\skola\code\New folder\envs\ann2test\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=951
  return f(*args, **kwargs)
E:\skola\code\New folder\envs\ann2test\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=710
  return f(*args, **kwargs)
E:\skola\code\New folder\envs\ann2test\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=698
  return f(*args, **kwargs)
E:\skola\code\New folder\envs\ann2test\lib\site-packages\librosa\util\decorators

In [183]:
def singleLine(data):

    feature = []
    for i in range(0, len(data)):
        feature.append(np.concatenate((
            data[i][0],
            data[i][1], 
            data[i][2], 
            data[i][3],
            data[i][4]), axis=0))
    array = np.array(feature)
    return array

In [191]:
X_train = singleLine(train_features)
X_test = singleLine(test_features)
X_val = singleLine(val_features)
y_train = np.array(train_data['speaker'])
y_val = np.array(val_data['speaker'])
y_test = np.array(test_data['speaker'])

In [192]:
# Hot encoding y
lb = LabelEncoder()
y_train = to_categorical(lb.fit_transform(y_train))
y_val = to_categorical(lb.fit_transform(y_val))

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_val = ss.transform(X_val)
X_test = ss.transform(X_test)

In [193]:

# Build a simple dense model with early stopping and softmax for categorical classification, remember we have 30 classes
model = Sequential()
model.add(Dense(193, input_shape=(193,), activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(27, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')

In [194]:
history = model.fit(X_train, y_train, batch_size=256, epochs=100, 
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop])

Epoch 1/100


ValueError: in user code:

    File "E:\skola\code\New folder\envs\ann2test\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "E:\skola\code\New folder\envs\ann2test\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "E:\skola\code\New folder\envs\ann2test\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "E:\skola\code\New folder\envs\ann2test\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "E:\skola\code\New folder\envs\ann2test\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "E:\skola\code\New folder\envs\ann2test\lib\site-packages\keras\engine\compile_utils.py", line 201

In [175]:
# Check out our train accuracy and validation accuracy over epochs.
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
# Set figure size.
plt.figure(figsize=(12, 8))
# Generate line plot of training, testing loss over epochs.
plt.plot(train_accuracy, label='Training Accuracy', color='#185fad')
plt.plot(val_accuracy, label='Validation Accuracy', color='orange')
# Set title
plt.title('Training and Validation Accuracy by Epoch', fontsize = 25)
plt.xlabel('Epoch', fontsize = 18)
plt.ylabel('Categorical Crossentropy', fontsize = 18)
plt.xticks(range(0,100,5), range(0,100,5))
plt.legend(fontsize = 18)

NameError: name 'history' is not defined